In [1]:
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel('./data/data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [4]:
X = data.drop('price', axis=1)
y = data['price']

---

### Работа с кодированием

> Произведем кодирование категориальных признаков с использованием методов Scikit-learn

In [5]:
# Пример работы кодировщика
lb = LabelBinarizer()

education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
lb.fit(education)
print('категории:', lb.classes_)

lb.transform(['нет', 'MSc'])


категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

In [6]:
# Кодирование признаков датасета
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']

for column in columns_to_change:
    print('Число уникальных значений признака {}: '.format(
        column), data[column].nunique())


Число уникальных значений признака cylinders:  6
Число уникальных значений признака title_status:  5
Число уникальных значений признака transmission:  3
Число уникальных значений признака drive:  3
Число уникальных значений признака size:  4


In [7]:
one_hot_encoder = OneHotEncoder()

# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()

# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)


['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


In [8]:
# Создадим таблицу кодировок
data_onehot = pd.DataFrame(data=data_onehot, columns=column_names, index=data.index)

In [9]:
# Объединяем таблицу кодировок с первичным датафреймом
data_new = pd.concat([data, data_onehot], axis=1)

In [10]:
# Удаляем столбцы которые были базой для onehot
data_new = data_new.drop(columns_to_change, axis=1)

In [11]:
# Смотрим форму полученного датафрейма
data_new.shape

(7017, 30)

---

#### Работа с пропусками

In [12]:
# Посмотрим пропуски в таблице
data.isna().sum()

price              0
year               0
condition          0
cylinders          0
odometer           0
title_status       0
transmission       0
drive            391
size            1564
lat                0
long               0
weather          180
dtype: int64

In [13]:
# Посмотрим пропуски в таблице
data.isna().mean()*100

price            0.000000
year             0.000000
condition        0.000000
cylinders        0.000000
odometer         0.000000
title_status     0.000000
transmission     0.000000
drive            5.572182
size            22.288727
lat              0.000000
long             0.000000
weather          2.565199
dtype: float64

In [14]:
data[~data['weather'].isna()]

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
5,6995,2003,3,8,167662,clean,automatic,4wd,full-size,45.518031,-122.578752,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,6,23500,clean,automatic,rwd,full-size,32.680700,-117.169800,59.0
7013,5975,2005,2,8,0,clean,automatic,rwd,full-size,38.213303,-85.785762,50.0
7014,9999,2006,3,8,161514,clean,automatic,NaN,full-size,37.609783,-120.995406,59.0
7015,10900,2011,2,8,164000,clean,automatic,4wd,full-size,43.140600,-93.385000,47.0


In [15]:
X = X.dropna() # Дропнем на матрице наблюдений
y = y.iloc[X.index] # И на векторе ответов

In [16]:
# Разделим на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [17]:
# Проведем кодирование
one_hot_encoder = OneHotEncoder()

X_train_enc = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray() # на тренировочной выборке используем фит_трансформ
X_test_enc = one_hot_encoder.transform(X_test[columns_to_change]).toarray() # а на тестовой только трансформ

In [18]:
columns = one_hot_encoder.get_feature_names_out()

In [19]:
X_train_onehot_df = pd.DataFrame(X_train_enc, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_enc, columns=columns)

In [20]:
# Сбросим индексы после разбиения на трейн и тест
X_train = X_train.reset_index().drop(['index'], axis=1)
X_test = X_test.reset_index().drop(['index'], axis=1)

y_train = y_train.reset_index().drop(['index'], axis=1)
y_test = y_test.reset_index().drop(['index'], axis=1)


In [21]:
# Объединяем таблицы и удаляем старые категориальные признаки
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)

X_train_new = X_train_new.drop(columns=columns_to_change)
X_test_new = X_test_new.drop(columns=columns_to_change)


In [22]:
# Настало время обучить модель. Для этого создаём объект класса LinearRegression.
lr_model = linear_model.LinearRegression()

In [23]:
# Обучаем модель по МНК
lr_model.fit(X_train_new, y_train)

LinearRegression()

In [24]:
# Делаем предсказание для тренировочной выборки
y_train_predict = lr_model.predict(X_train_new)

In [25]:
# Делаем предсказание для тестовой выборки:

y_test_predict = lr_model.predict(X_test_new)
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict)))

Train R^2: 0.647
Test R^2: 0.693


---

#### Заполнение пропусков

> Среднее

In [26]:
# Использование среднего значения
X_train_new['weather'] = X_train_new['weather'].fillna(
    np.round(np.mean(X_train_new['weather']), 0))

X_test['weather'] = X_test_new['weather'].fillna(
    np.round(np.mean(X_train_new['weather']), 0))


> Мода

In [27]:
X_train['drive'].value_counts(True).head(1)

4wd    0.736602
Name: drive, dtype: float64

In [28]:
X_train['size'].value_counts(True).head(1)

full-size    0.830089
Name: size, dtype: float64

In [29]:
X_train['drive'] = X_train['drive'].fillna(X_train['drive'].mode())

X_test['drive'] = X_test['drive'].fillna(X_train['drive'].mode())

X_train['size'] = X_train['size'].fillna(X_train['size'].mode())

X_test['size'] = X_test['size'].fillna(X_train['size'].mode())


#### Замена пропусков через предсказание модели

In [30]:
data = pd.read_excel('./data/data_ford_price.xlsx')

X = data.drop('price', axis=1)
y = data['price']

In [31]:
X.head()

,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [32]:
# Заполнение пропусков модельным способом 

weather = X.copy() # копируем матрицу наблюдений

test_data = weather[weather['weather'].isnull()] # создаем промежуточный фрейм включающий все пропуски по указаному фиту
weather.dropna(inplace=True) # удаляем пропуски из первичной матрицы

y_train = weather['weather'] # создаем тренировочный вектор ответов на безнулевом датафрейме
X_train = weather.drop(['size', 'weather', 'drive'], axis=1) # создаем матрицу наблюдений на безнулевом датафрейме
X_test = test_data.drop(['size', 'weather', 'drive'], axis=1) # создаем тестовую матрицу на основе фрейма пропусков удалив все фиты в которых изначально были nan

one_hot_encoder = OneHotEncoder() # создаем объект кодировщика
categorial_cols = ['cylinders', 'title_status', 'transmission'] # создаем список фитов для кодировки

X_train_onehot = one_hot_encoder.fit_transform(
    X_train[categorial_cols]).toarray()  
X_test_onehot = one_hot_encoder.transform(X_test[categorial_cols]).toarray() # кодируем

columns = one_hot_encoder.get_feature_names_out(categorial_cols) # берем наименовая фитов для создания датафреймов
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns) # создаем трэйн и тест датафреймы закодированных признаков
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

X_train = X_train.reset_index().drop(['index'], axis=1) # сбрасываем индексы и удаляем ненужную колонку
X_test = X_test.reset_index().drop(['index'], axis=1)
y_train = y_train.reset_index().drop(['index'], axis=1)

X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1) # объединяем 
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)

X_train_new = X_train_new.drop(columns=categorial_cols)
X_test_new = X_test_new.drop(columns=categorial_cols)


model = linear_model.LinearRegression()
model.fit(X_train_new, y_train)

y_pred = model.predict(X_test_new)
y_pred.shape

(180, 1)

---

#### Своя практика

In [33]:
# Инициализируем датафрейм и разобьем его на X и y по целевому признаку
data = pd.read_excel('./data/data_ford_price.xlsx')

X = data.drop('price', axis=1)
y = data['price']

# Создадим список фитов с пропусками
# получим список строковых обозначений фитов
list_of_empty = [i for i in X.isna().sum().index if X[i].isna().sum() > 0]

# Создадим копии первичного фрейма для заполнения пропусков
weather = X.copy()  # числовой признак будем использовать линейную регрессию
size = X.copy()  # категориальный будем использовать логистическую
drive = X.copy()  # категориальный

# Создадим блок фреймов наполненных пропусками по целевым колонкам
test_weather_data = weather[weather['weather'].isna()]
test_size_data = size[size['size'].isna()]
test_drive_data = drive[drive['drive'].isna()]

# Удалим все пропуски
weather.dropna(inplace=True)
size.dropna(inplace=True)
drive.dropna(inplace=True)

# Для каждого фита создадим трейн/тест, скинув на трейне фиты с пустотами
y_train_weather = weather['weather']
# здесь скидываем и пустые фиты и целевой фит
X_train_weather = weather.drop(list_of_empty, axis=1)
X_test_weather = test_weather_data.drop(list_of_empty, axis=1)

y_train_size = size['size']
X_train_size = size.drop(list_of_empty, axis=1)
X_test_size = test_size_data.drop(list_of_empty, axis=1)

y_train_drive = drive['drive']
X_train_drive = drive.drop(list_of_empty, axis=1)
X_test_drive = test_drive_data.drop(list_of_empty, axis=1)

# Приступим к кодированию категориальных признаков через one hot для увеличения предсказательной способностей модели
one_hot_encoder_weather = OneHotEncoder()  # создаем объект кодировщика
one_hot_encoder_size = OneHotEncoder()
one_hot_encoder_drive = OneHotEncoder()

# создаем список фитов для кодировки
categorial_cols = ['cylinders', 'title_status', 'transmission']

# Закодируем блок
X_train_weather_oh = one_hot_encoder_weather.fit_transform(
    X_train_weather[categorial_cols]).toarray()
X_test_weather_oh = one_hot_encoder_weather.transform(
    X_test_weather[categorial_cols]).toarray()

X_train_size_oh = one_hot_encoder_size.fit_transform(
    X_train_size[categorial_cols]).toarray()
X_test_size_oh = one_hot_encoder_size.transform(
    X_test_size[categorial_cols]).toarray()

X_train_drive_oh = one_hot_encoder_drive.fit_transform(
    X_train_drive[categorial_cols]).toarray()
X_test_drive_oh = one_hot_encoder_drive.transform(
    X_test_drive[categorial_cols]).toarray()

# Создадим из полученных закодированных фитов датафреймы
# т.к. у нас закодированы одни и те же категориальные фиты мы можем получить имена колонок из любого кодировщика
columns = one_hot_encoder_weather.get_feature_names_out()

X_train_weather_oh_df = pd.DataFrame(data=X_train_weather_oh, columns=columns)
X_test_weather_oh_df = pd.DataFrame(data=X_test_weather_oh, columns=columns)

X_train_size_oh_df = pd.DataFrame(data=X_train_size_oh, columns=columns)
X_test_size_oh_df = pd.DataFrame(data=X_test_size_oh, columns=columns)

X_train_drive_oh_df = pd.DataFrame(data=X_train_drive_oh, columns=columns)
X_test_drive_oh_df = pd.DataFrame(data=X_test_drive_oh, columns=columns)

# Сбросим индексы у извлеченных выборок т.к они пришли из основного датафрейма
y_train_weather = y_train_weather.reset_index().drop(['index'], axis=1)
X_train_weather = X_train_weather.reset_index().drop(['index'], axis=1)
X_test_weather = X_test_weather.reset_index().drop(['index'], axis=1)

y_train_size = y_train_size.reset_index().drop(['index'], axis=1)
X_train_size = X_train_size.reset_index().drop(['index'], axis=1)
X_test_size = X_test_size.reset_index().drop(['index'], axis=1)

y_train_drive = y_train_drive.reset_index().drop(['index'], axis=1)
X_train_drive = X_train_drive.reset_index().drop(['index'], axis=1)
X_test_drive = X_test_drive.reset_index().drop(['index'], axis=1)

# Объединим фреймы
X_train_weather_new = pd.concat(
    [X_train_weather, X_train_weather_oh_df], axis=1)
X_test_weather_new = pd.concat([X_test_weather, X_test_weather_oh_df], axis=1)

X_train_size_new = pd.concat([X_train_size, X_train_size_oh_df], axis=1)
X_test_size_new = pd.concat([X_test_size, X_test_size_oh_df], axis=1)

X_train_drive_new = pd.concat([X_train_drive, X_train_drive_oh_df], axis=1)
X_test_drive_new = pd.concat([X_test_drive, X_test_drive_oh_df], axis=1)

# Удалим категориальные фиты, т.к. мы их уже закодировали
X_train_weather_new = X_train_weather_new.drop(columns=categorial_cols, axis=1)
X_test_weather_new = X_test_weather_new.drop(columns=categorial_cols, axis=1)

X_train_size_new = X_train_size_new.drop(columns=categorial_cols, axis=1)
X_test_size_new = X_test_size_new.drop(columns=categorial_cols, axis=1)

X_train_drive_new = X_train_drive_new.drop(columns=categorial_cols, axis=1)
X_test_drive_new = X_test_drive_new.drop(columns=categorial_cols, axis=1)

# Произведем моделирование linear regression для числовых, logistic regression для категориальных
weather_model = linear_model.LinearRegression()
weather_model.fit(X_train_weather_new, y_train_weather)
y_test_weather = weather_model.predict(X_test_weather_new)

size_model = linear_model.LogisticRegression(
    class_weight='balanced', random_state=42, max_iter=500, n_jobs=-1)
size_model.fit(X_train_size_new, y_train_size['size'].ravel())
y_test_size = size_model.predict(X_test_size_new)

drive_model = linear_model.LogisticRegression(
    class_weight='balanced', random_state=42, max_iter=500, n_jobs=-1)
drive_model.fit(X_train_drive_new, y_train_drive['drive'].ravel())
y_test_drive = drive_model.predict(X_test_drive_new)

# Запихнем результаты заполнений в датафрейм
X.loc[X['weather'].isna(), 'weather'] = y_test_weather
X.loc[X['size'].isna(), 'size'] = y_test_size
X.loc[X['drive'].isna(), 'drive'] = y_test_drive


---

#### Работа с выбросами через модели

In [34]:
# Инициализируем датафрейм и разобьем его на X и y по целевому признаку
data = pd.read_excel('./data/data_ford_price.xlsx')

X = data.drop('price', axis=1)
y = data['price']

In [35]:
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


Для начала сформируем baseline-модель. Проведём следующую предобработку: для простоты уберём категориальные столбцы из данных и затем удалим строки с пропусками.

In [36]:
def outliers_z_score(data, feature, left_mod=3, right_mod=3, log_scale=False):
    """Function for finding outliers by z-method with adjustment option for left and right multiplier.

    Args:
        data (DataFrame): DataFrame which will be used to find outliers.
        feature (string): Name of a column in DF which will be inspected for outliers.
        left_mod (int, optional): lower boundary multiplier. Defaults to 3.
        right_mod (int, optional): upper boundary multiplier. Defaults to 3.
        log_scale (bool, optional): converting data in logarithmic representation in case of lognormal destribution of
        original data. Defaults to False.

    Returns:
        DataFrame: returns two copies of the original DataFrame contain DF's with outliers and cleaned data.
    """    ''''''

    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]

    mu = x.mean()

    sigma = x.std()

    lower_bound = mu - left_mod * sigma

    upper_bound = mu + right_mod * sigma

    outliers = data[(x < lower_bound) | (x > upper_bound)]

    cleaned = data[(x > lower_bound) & (x < upper_bound)]

    print(f'Number of outliers by z-method: {outliers.shape[0]}')
    print(f'Resulting number of lines cleared of outliers: {cleaned.shape[0]}')

    return outliers, cleaned

In [37]:
_, data = outliers_z_score(data, 'price')
_, data = outliers_z_score(data, 'year')
_, data = outliers_z_score(data, 'odometer')

Number of outliers by z-method: 41
Resulting number of lines cleared of outliers: 6976
Number of outliers by z-method: 85
Resulting number of lines cleared of outliers: 6891
Number of outliers by z-method: 17
Resulting number of lines cleared of outliers: 6874


In [38]:
data = data[[
    'price', 'year', 'cylinders',
    'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace=True)

y = data['price']
X = data.drop(columns='price')
X.head()


,year,cylinders,odometer,lat,long,weather
0,2016,6,43500,36.471500,-82.483400,59.0
1,2009,8,98131,40.468826,-74.281734,52.0
2,2002,8,201803,42.477134,-82.949564,45.0
3,2000,8,170305,40.764373,-82.349503,49.0
5,2003,8,167662,45.518031,-122.578752,50.0


In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=30)

model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

mae = metrics.mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)


MAE: 4045.733


Первый алгоритм, который мы применим, - Isolation Forest, или iForest. Это алгоритм обнаружения аномалий на основе дерева.
Данный метод стремится изолировать аномалии, которые немногочисленны и различаются по пространству признаков.

Библиотека scikit-learn предоставляет реализацию Isolation Forest в классе IsolationForest.
Одним из основных гиперпараметров модели является contamination («загрязнение»), который используется для оценки количества
выбросов в наборе данных. Его значение находится в диапазоне от 0.0 до 0.5 и по умолчанию равно 0.1.

In [40]:
from sklearn.ensemble import IsolationForest

# ищем выбросы в обучающей выборке
iso = IsolationForest(contamination=0.1)
y_predicted_iso = iso.fit_predict(X_train)

# выберем все строки, которые не являются выбросами
mask = y_predicted_iso != -1
X_train_iso, y_train_iso = X_train[mask], y_train[mask]

print(X_train.shape, y_train.shape)

model = linear_model.LinearRegression()
model.fit(X_train_iso, y_train_iso)

y_predicted = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)


/Users/kirillkuznecov/opt/anaconda3/envs/ds/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


(4688, 6) (4688,)
MAE: 4051.904


Следующий метод - Local Outlier Factor, или LOF. Это метод, который пытается использовать идею ближайших соседей для
обнаружения выбросов.

Каждому примеру присваивается оценка того, насколько он изолирован от его локальных соседей. Примеры, которые наиболее
отдалены от соседей, скорее всего, будут являться выбросами.

Библиотека scikit-learn обеспечивает реализацию этого подхода в классе LocalOutlierFactor.

In [41]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor()
y_predicted_lof = lof.fit_predict(X_train)

mask = y_predicted_lof != -1
X_train_lof, y_train_lof = X_train[mask], y_train[mask]

print(X_train_lof.shape, y_train_lof.shape)

model = linear_model.LinearRegression()
model.fit(X_train_lof, y_train_lof)

y_predicted = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)


(4330, 6) (4330,)
MAE: 4040.503


Напоследок рассмотрим Minimum Covariance Determinant, или MCD.
Если входные переменные имеют гауссово распределение, то для обнаружения выбросов можно использовать простые статистические методы.
Например, если набор данных имеет две входные переменные и обе они являются гауссовыми, то пространство признаков образует
многомерную гауссовскую зависимость, и знание этого распределения можно использовать для определения значений, далёких от
распределения.

Этот подход можно обобщить, определив гиперсферу (эллипсоид), которая покрывает нормальные данные, а данные, выходящие за
пределы этой формы, считаются выбросами. Эффективная реализация этого метода для многомерных данных известна как
детерминант минимальной ковариации (MCD).

Библиотека scikit-learn предоставляет доступ к этому методу через класс EllipticEnvelope.

In [42]:
from sklearn.covariance import EllipticEnvelope

ee = EllipticEnvelope(contamination=0.01)
y_predicted_ee = ee.fit_predict(X_train)

mask = y_predicted_ee != -1
X_train_ee, y_train_ee = X_train[mask], y_train[mask]

print(X_train_ee.shape, y_train_ee.shape)

model = linear_model.LinearRegression()
model.fit(X_train, y_train)

y_predicted = model.predict(X_test)
mae = metrics.mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)


(4641, 6) (4641,)
MAE: 4045.733


---

In [43]:
# Инициализируем датафрейм и разобьем его на X и y по целевому признаку
from sklearn import preprocessing
data = pd.read_excel('./data/data_ford_price.xlsx')


In [44]:
data = data[['price', 'year', 'weather']]
data.dropna(inplace=True)
data_scaled = data.copy()
col_names = ['price', 'weather']
x = data_scaled[col_names]


scaler = preprocessing.RobustScaler()

data_scaled[col_names] = scaler.fit_transform(x.values)
data_scaled


,price,year,weather
0,1.982455,2016,0.571429
1,0.189460,2009,0.071429
2,-0.630672,2002,-0.428571
3,-0.706090,2000,-0.142857
5,-0.346671,2003,-0.071429
...,...,...,...
7012,0.631871,2015,0.571429
7013,-0.411044,2005,-0.071429
7014,-0.157084,2006,0.571429
7015,-0.100221,2011,-0.285714


---